# XGBoost Model - Implementation

## Features

- **XGBoostModel**: Single-target XGBoost regressor
- **XGBoostGoalPredictor**: Dual model for home/away goals
- Feature importance analysis
- SHAP value interpretation
- Cross-validation support
- Early stopping

## Hyperparameters

| Parameter | Default | Range | Impact |
|-----------|---------|-------|--------|
| learning_rate | 0.05 | 0.01-0.3 | Lower = more trees needed |
| n_estimators | 500 | 100-2000 | More = slower but better |
| max_depth | 6 | 3-10 | Higher = more complex |
| min_child_weight | 3 | 1-10 | Higher = more regularization |
| subsample | 0.8 | 0.5-1.0 | Stochastic training |
| colsample_bytree | 0.8 | 0.5-1.0 | Feature sampling |

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Check XGBoost availability
try:
    import xgboost as xgb
    print(f"XGBoost version: {xgb.__version__}")
except ImportError:
    print("XGBoost not installed. Run: pip install xgboost")

XGBoost not installed. Run: pip install xgboost


## XGBoostModel Class

In [2]:
class XGBoostModel:
    """
    XGBoost regression model for hockey goal prediction.
    
    Parameters
    ----------
    params : dict
        XGBoost hyperparameters
    scale_features : bool
        Whether to standardize features
    """
    
    DEFAULT_PARAMS = {
        'learning_rate': 0.05,
        'n_estimators': 500,
        'max_depth': 6,
        'min_child_weight': 3,
        'gamma': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.1,
        'reg_lambda': 1.0,
        'objective': 'reg:squarederror',
        'random_state': 42,
        'n_jobs': -1,
        'tree_method': 'hist',
    }
    
    def __init__(self, params=None, scale_features=False):
        self.params = {**self.DEFAULT_PARAMS, **(params or {})}
        self.scale_features = scale_features
        self.scaler = None
        self.model = None
        self.feature_names = None
        self.feature_importances_ = None
        self.is_fitted = False
        self.training_history = None
    
    def fit(self, X, y, X_val=None, y_val=None, early_stopping_rounds=50, verbose=False):
        """
        Train the XGBoost model.
        
        Parameters
        ----------
        X : pd.DataFrame
            Training features
        y : pd.Series
            Training targets
        X_val, y_val : optional
            Validation data for early stopping
        """
        # Store feature names
        if isinstance(X, pd.DataFrame):
            self.feature_names = list(X.columns)
        
        # Optional scaling
        if self.scale_features:
            from sklearn.preprocessing import StandardScaler
            self.scaler = StandardScaler()
            X = self.scaler.fit_transform(X)
            if X_val is not None:
                X_val = self.scaler.transform(X_val)
        
        # Create and fit model
        self.model = xgb.XGBRegressor(**self.params)
        
        if X_val is not None and y_val is not None:
            self.model.fit(
                X, y,
                eval_set=[(X, y), (X_val, y_val)],
                verbose=verbose
            )
            self.training_history = self.model.evals_result()
        else:
            self.model.fit(X, y, verbose=verbose)
        
        # Store feature importances
        if self.feature_names:
            self.feature_importances_ = pd.Series(
                self.model.feature_importances_,
                index=self.feature_names
            ).sort_values(ascending=False)
        
        self.is_fitted = True
        return self
    
    def predict(self, X):
        """Predict target values."""
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted first")
        
        if self.scale_features and self.scaler:
            X = self.scaler.transform(X)
        
        return self.model.predict(X)
    
    def evaluate(self, X, y):
        """Evaluate model performance."""
        preds = self.predict(X)
        return {
            'rmse': np.sqrt(mean_squared_error(y, preds)),
            'mae': mean_absolute_error(y, preds),
            'r2': r2_score(y, preds)
        }
    
    def plot_feature_importance(self, top_n=15):
        """Plot top feature importances."""
        if self.feature_importances_ is None:
            raise RuntimeError("Model must be fitted first")
        
        top_features = self.feature_importances_.head(top_n)
        
        fig, ax = plt.subplots(figsize=(10, 8))
        top_features.sort_values().plot(kind='barh', ax=ax, color='steelblue')
        ax.set_xlabel('Importance')
        ax.set_title(f'Top {top_n} Feature Importances')
        plt.tight_layout()
        return fig
    
    def plot_training_history(self):
        """Plot training and validation loss."""
        if self.training_history is None:
            raise RuntimeError("No training history - fit with validation set")
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        for name, values in self.training_history.items():
            for metric, scores in values.items():
                label = f"{name} - {metric}"
                ax.plot(scores, label=label)
        
        ax.set_xlabel('Iteration')
        ax.set_ylabel('RMSE')
        ax.set_title('Training History')
        ax.legend()
        plt.tight_layout()
        return fig

## XGBoostGoalPredictor Class

Wrapper that uses two XGBoost models to predict home and away goals separately.

In [3]:
class XGBoostGoalPredictor:
    """
    High-level wrapper for predicting both home and away goals.
    
    Uses separate XGBoost models for home and away goals.
    """
    
    def __init__(self, params=None):
        self.params = params or {}
        self.home_model = XGBoostModel(params)
        self.away_model = XGBoostModel(params)
        self.feature_columns = None
        self.is_fitted = False
    
    def fit(self, games_df, feature_columns=None):
        """
        Train both models on game data.
        
        Parameters
        ----------
        games_df : pd.DataFrame
            Must have home_goals, away_goals, and feature columns
        feature_columns : list, optional
            Columns to use as features. Auto-detected if None.
        """
        # Identify feature columns
        if feature_columns is None:
            exclude = {'home_goals', 'away_goals', 'home_team', 'away_team', 'date', 'game_id'}
            feature_columns = [
                col for col in games_df.columns 
                if col not in exclude and games_df[col].dtype in ['int64', 'float64']
            ]
        
        self.feature_columns = feature_columns
        X = games_df[feature_columns]
        
        # Train both models
        self.home_model.fit(X, games_df['home_goals'])
        self.away_model.fit(X, games_df['away_goals'])
        
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        """Predict home and away goals for a single game."""
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted first")
        
        if isinstance(game, dict):
            game = pd.Series(game)
        
        X = game[self.feature_columns].values.reshape(1, -1)
        
        home_pred = self.home_model.predict(X)[0]
        away_pred = self.away_model.predict(X)[0]
        
        return float(home_pred), float(away_pred)
    
    def predict_batch(self, games_df):
        """Predict goals for multiple games."""
        X = games_df[self.feature_columns]
        return pd.DataFrame({
            'home_pred': self.home_model.predict(X),
            'away_pred': self.away_model.predict(X)
        })
    
    def evaluate(self, games_df):
        """Evaluate both models on test set."""
        X = games_df[self.feature_columns]
        
        home_metrics = self.home_model.evaluate(X, games_df['home_goals'])
        away_metrics = self.away_model.evaluate(X, games_df['away_goals'])
        
        # Combined metrics
        all_preds = np.concatenate([
            self.home_model.predict(X),
            self.away_model.predict(X)
        ])
        all_actual = np.concatenate([
            games_df['home_goals'].values,
            games_df['away_goals'].values
        ])
        
        return {
            'home': home_metrics,
            'away': away_metrics,
            'combined': {
                'rmse': np.sqrt(mean_squared_error(all_actual, all_preds)),
                'mae': mean_absolute_error(all_actual, all_preds),
                'r2': r2_score(all_actual, all_preds)
            }
        }

## Demo with Synthetic Data

In [4]:
# Generate synthetic hockey game data
np.random.seed(42)
n_games = 1000

# Create feature dataframe
data = pd.DataFrame({
    'home_win_pct': np.random.uniform(0.3, 0.7, n_games),
    'away_win_pct': np.random.uniform(0.3, 0.7, n_games),
    'home_goals_avg': np.random.uniform(2.5, 3.5, n_games),
    'away_goals_avg': np.random.uniform(2.5, 3.5, n_games),
    'home_goals_against_avg': np.random.uniform(2.5, 3.5, n_games),
    'away_goals_against_avg': np.random.uniform(2.5, 3.5, n_games),
    'home_pp_pct': np.random.uniform(0.15, 0.25, n_games),
    'away_pp_pct': np.random.uniform(0.15, 0.25, n_games),
    'home_pk_pct': np.random.uniform(0.75, 0.85, n_games),
    'away_pk_pct': np.random.uniform(0.75, 0.85, n_games),
    'home_rest_days': np.random.randint(1, 5, n_games),
    'away_rest_days': np.random.randint(1, 5, n_games),
})

# Generate realistic goal totals
home_advantage = 0.3
data['home_goals'] = np.round(
    data['home_goals_avg'] * 0.5 +
    (4 - data['away_goals_against_avg']) * 0.5 +
    home_advantage +
    np.random.normal(0, 0.7, n_games)
).clip(0, 8).astype(int)

data['away_goals'] = np.round(
    data['away_goals_avg'] * 0.5 +
    (4 - data['home_goals_against_avg']) * 0.5 +
    np.random.normal(0, 0.7, n_games)
).clip(0, 8).astype(int)

print(f"Generated {n_games} synthetic games")
print(f"\nHome goals distribution:")
print(data['home_goals'].value_counts().sort_index())

Generated 1000 synthetic games

Home goals distribution:
home_goals
0      4
1    123
2    473
3    358
4     42
Name: count, dtype: int64


In [5]:
# Split data
train_size = int(0.8 * n_games)
train_df = data[:train_size]
test_df = data[train_size:]

print(f"Training set: {len(train_df)} games")
print(f"Test set: {len(test_df)} games")

Training set: 800 games
Test set: 200 games


In [6]:
# Train XGBoostGoalPredictor
predictor = XGBoostGoalPredictor({
    'max_depth': 4,
    'n_estimators': 200,
    'learning_rate': 0.1
})

predictor.fit(train_df)

# Evaluate
metrics = predictor.evaluate(test_df)

print("\n📊 Test Set Performance")
print("=" * 40)
print(f"\nHome Goals:")
print(f"  RMSE: {metrics['home']['rmse']:.4f}")
print(f"  MAE:  {metrics['home']['mae']:.4f}")
print(f"  R²:   {metrics['home']['r2']:.4f}")

print(f"\nAway Goals:")
print(f"  RMSE: {metrics['away']['rmse']:.4f}")
print(f"  MAE:  {metrics['away']['mae']:.4f}")
print(f"  R²:   {metrics['away']['r2']:.4f}")

print(f"\nCombined:")
print(f"  RMSE: {metrics['combined']['rmse']:.4f}")
print(f"  MAE:  {metrics['combined']['mae']:.4f}")
print(f"  R²:   {metrics['combined']['r2']:.4f}")

NameError: name 'xgb' is not defined

In [ ]:
# Feature importance
fig = predictor.home_model.plot_feature_importance(top_n=10)
plt.title('Feature Importance (Home Goals Model)')
plt.show()

In [ ]:
# Prediction vs Actual scatter plot
predictions = predictor.predict_batch(test_df)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Home goals
axes[0].scatter(test_df['home_goals'], predictions['home_pred'], alpha=0.5)
axes[0].plot([0, 8], [0, 8], 'r--', label='Perfect prediction')
axes[0].set_xlabel('Actual Home Goals')
axes[0].set_ylabel('Predicted Home Goals')
axes[0].set_title('Home Goals: Predicted vs Actual')
axes[0].legend()

# Away goals
axes[1].scatter(test_df['away_goals'], predictions['away_pred'], alpha=0.5)
axes[1].plot([0, 8], [0, 8], 'r--', label='Perfect prediction')
axes[1].set_xlabel('Actual Away Goals')
axes[1].set_ylabel('Predicted Away Goals')
axes[1].set_title('Away Goals: Predicted vs Actual')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis
residuals_home = test_df['home_goals'] - predictions['home_pred']
residuals_away = test_df['away_goals'] - predictions['away_pred']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(residuals_home, bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='red', linestyle='--')
axes[0].set_xlabel('Residual (Actual - Predicted)')
axes[0].set_ylabel('Count')
axes[0].set_title(f'Home Goals Residuals (mean: {residuals_home.mean():.3f})')

axes[1].hist(residuals_away, bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[1].axvline(0, color='red', linestyle='--')
axes[1].set_xlabel('Residual (Actual - Predicted)')
axes[1].set_ylabel('Count')
axes[1].set_title(f'Away Goals Residuals (mean: {residuals_away.mean():.3f})')

plt.tight_layout()
plt.show()

## Single Game Prediction

In [ ]:
# Predict a single game
sample_game = {
    'home_win_pct': 0.65,
    'away_win_pct': 0.45,
    'home_goals_avg': 3.2,
    'away_goals_avg': 2.8,
    'home_goals_against_avg': 2.5,
    'away_goals_against_avg': 3.0,
    'home_pp_pct': 0.22,
    'away_pp_pct': 0.18,
    'home_pk_pct': 0.82,
    'away_pk_pct': 0.78,
    'home_rest_days': 2,
    'away_rest_days': 1,
}

home_pred, away_pred = predictor.predict_goals(sample_game)

print("🏒 Game Prediction")
print("=" * 40)
print(f"Home team predicted goals: {home_pred:.2f}")
print(f"Away team predicted goals: {away_pred:.2f}")
print(f"\nPredicted winner: {'Home' if home_pred > away_pred else 'Away'} team")
print(f"Expected total goals: {home_pred + away_pred:.2f}")

## Save Model

In [ ]:
import os

# Create output directory
os.makedirs('../output/models', exist_ok=True)

# Save both models
predictor.home_model.model.save_model('../output/models/xgboost_home.json')
predictor.away_model.model.save_model('../output/models/xgboost_away.json')

print(" Models saved to ../output/models/")
print("   - xgboost_home.json")
print("   - xgboost_away.json")

## Summary

The XGBoost model provides:

1. **High accuracy** - Industry-standard gradient boosting
2. **Feature importance** - Understand what drives predictions
3. **Flexibility** - Many hyperparameters to tune
4. **Speed** - Histogram-based training is very fast
5. **Regularization** - Built-in L1/L2 regularization

### Next Steps

- See `training/train_xgboost.ipynb` for hyperparameter tuning
- See `validation/validate_xgboost.ipynb` for comprehensive testing
- Import from `utils.xgboost_model` for production use